# Model Performance

We will analyze the 2 Yolo models supplied and compare their performance

<b>Pre-requisites</b>: The notebook assumes that you have ffmpeg installed on the container or host where this notebook is running. If this is running on a container then make sure the container's docker file has the following 

apt-get update -qq && apt-get install ffmpeg -y

In [ ]:
!pip install numpy 
!pip install matplotlib
!pip uninstall -y opencv-python-headless
!pip uninstall -y opencv-python
!pip uninstall -y cv2
!pip uninstall -y pylabel
!pip install pylabel
!pip install opencv-python-headless
!pip install pytesseract
!pip install ffmpeg-python
!pip install pandas 

We first do a setup here to enable this notebook to access classes in the parent directory

In [1]:
# Set up the notebook to import modules from relative paths
import os, sys

#'/home/user/example/parent/child'
current_path = os.path.abspath('.')

#'/home/user/example/parent'
parent_path = os.path.dirname(current_path)

sys.path.append(parent_path)

We have already loaded the images from the video stream, cropped them and stored the cropped images in an external volume mapped to the current docker file system. In this notebook we will make use of the Object_Detection_Dataset to get a testing dataset to determine which model is performing better. 

For context we have 2 models with the following files

<ul>
    <li>lpr-yolov3.cfg and lpr-yolov3.weights</li>
    <li>lpr-yolov3-tiny.cfg and lpr-yolov3-tiny.weights</li>
</ul>

All the images essentially show 2 license plates clearly and so the model that detects the more plates is likely to have higher precision

In [8]:
import numpy as np
import matplotlib.pyplot as plt 
import cv2 as cv
from dataset import Object_Detection_Dataset

folds = 10
dataset = Object_Detection_Dataset(data_folder='/workspace/shared-data/license-plates/alpr-images/cropped/',n_folds=folds)

In [9]:
from model import Object_Detection_Model

model1 = Object_Detection_Model(cfg_file= "lpr-yolov3.cfg", weights_file='lpr-yolov3.weights', base_dir='/workspace/shared-data/license-plates/alpr-images/')
model2 = Object_Detection_Model(cfg_file= "lpr-yolov3-tiny.cfg", weights_file='lpr-yolov3-tiny.weights', base_dir='/workspace/shared-data/license-plates/alpr-images/')

In [17]:
# Maintain list for scores by each fold
precision1_scores = []
precision2_scores = []

for fold in range(folds):
    # Let's keep count of TP
    tp1 = 0
    tp2 = 0

    # Get the test data
    test_data = dataset.get_testing_dataset(fold)
    
    # On an average each image is showing 2 cars with their license plates visible - so total is twice as the number of images
    total_count = len(test_data) * 2
    print(fold)
    for img_file in test_data:
        results1 = model1.test(img_file,list())
        results2 = model2.test(img_file,list())
    
        tp1 += len(results1)
        tp2 += len(results2)
    
    precision1 = tp1 / total_count
    precision2 = tp2 / total_count
        
    precision1_scores += [precision1]
    precision2_scores += [precision2]

0
1
2
3
4
5
6
7
8
9


In [20]:
import pandas as pd
from IPython.display import display, HTML

# Display Properties
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

model_stats = np.column_stack((range(1,folds+1), precision1_scores, precision2_scores))
model_stats_df = pd.DataFrame(model_stats, columns = ['K-Fold','Model 1 Precision','Model 2 Precision'])
display(HTML(model_stats_df.to_html())) 

,K-Fold,Model 1 Precision,Model 2 Precision
0,1.000,0.442,0.365
1,2.000,0.404,0.452
2,3.000,0.392,0.382
3,4.000,0.392,0.333
4,5.000,0.451,0.451
5,6.000,0.422,0.343
6,7.000,0.422,0.392
7,8.000,0.353,0.324
8,9.000,0.392,0.412
9,10.000,0.422,0.314


From the above model, it looks like Model 1 is doing slightly better in terms of its precisions as it is measured using number of objects detected with a confidence score greater than 0.5. Given that we don't have currently ground truths available the above is not able to detect False Positives. 